In [1]:
import gradio as gr
from langchain import PromptTemplate, LLMChain
from langchain.chat_models import ChatOpenAI
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.memory import ConversationBufferMemory

# STT 파일에서 텍스트 로드
with open('./stt_file.txt', 'r', encoding='utf-8') as file:
    stt_text = file.read()

# 텍스트를 작은 조각으로 분할
text_splitter = RecursiveCharacterTextSplitter(chunk_size=2000, chunk_overlap=100)
text_chunks = text_splitter.split_text(stt_text)

# 메모리 설정
memory = ConversationBufferMemory()

# Gradio에서의 대화 함수
def chat_with_product_info(user_input, chat_history=[]):
    # 템플릿 정의
    prompt_template = PromptTemplate(
        input_variables=["input"],
        template="제품 정보: {input}\n\n질문에 대한 간단한 요약 답변(3줄 이내):"
    )

    # LLMChain 생성
    chat_chain = LLMChain(
        llm=ChatOpenAI(model_name="gpt-4o-mini"),
        prompt=prompt_template,
        memory=memory
    )

    # 각 텍스트 조각에 대해 질문을 처리하고 답변을 생성
    response_summary = ""
    for chunk in text_chunks:
        combined_input = f"{chunk}\n질문: {user_input}"
        response = chat_chain.predict(input=combined_input)
        response_summary = response.strip()  # 마지막 조각의 답변을 사용하여 요약

        if response_summary:  # 첫 번째로 의미 있는 요약만 사용
            break
    
    # 대화 기록에 추가
    chat_history.append(("사용자", user_input))
    chat_history.append(("챗봇", response_summary.strip()))

    # 대화 내용을 보기 좋게 정리 (HTML 형식으로)
    formatted_chat_history = ""
    for role, message in chat_history:
        if role == "사용자":
            formatted_chat_history += f'<div style="text-align: right; color: gray;"><strong>{role}</strong>: {message}</div>'
        else:
            formatted_chat_history += f'<div style="text-align: left;"><strong>{role}</strong>: {message}</div>'
    
    return formatted_chat_history, chat_history, ""

# Gradio 인터페이스 설정
def create_interface():
    with gr.Blocks() as gr_interface:
        with gr.Tab("제품"):
            chat_history_state = gr.State([])
            with gr.Row():
                with gr.Column(scale=8):
                    chat_output = gr.HTML(label="채팅 기록", interactive=False)
                with gr.Column(scale=4):
                    user_input = gr.Textbox(label="제품에 대해 질문해보세요", placeholder="예: 제품의 효능은 무엇인가요?")
                    submit_button = gr.Button("제출")
                    submit_button.click(chat_with_product_info, inputs=[user_input, chat_history_state], outputs=[chat_output, chat_history_state, user_input])

    return gr_interface

# Gradio 인터페이스 실행
if __name__ == "__main__":
    gr_interface = create_interface()
    gr_interface.launch()


Running on local URL:  http://127.0.0.1:7860
IMPORTANT: You are using gradio version 3.48.0, however version 4.29.0 is available, please upgrade.
--------

To create a public link, set `share=True` in `launch()`.


c:\Users\Admin\miniconda3\envs\gumi_env310\lib\site-packages\langchain_core\_api\deprecation.py:139: LangChainDeprecationWarning: The class `ChatOpenAI` was deprecated in LangChain 0.0.10 and will be removed in 0.3.0. An updated version of the class exists in the langchain-openai package and should be used instead. To use it run `pip install -U langchain-openai` and import as `from langchain_openai import ChatOpenAI`.
  warn_deprecated(
c:\Users\Admin\miniconda3\envs\gumi_env310\lib\site-packages\langchain_core\_api\deprecation.py:139: LangChainDeprecationWarning: The class `LLMChain` was deprecated in LangChain 0.1.17 and will be removed in 1.0. Use RunnableSequence, e.g., `prompt | llm` instead.
  warn_deprecated(
